# Swin Depth Controller - Sudoku Training

Train the Swin-style depth controller on Sudoku-Extreme with Optuna search.

In [ ]:
# Install dependencies
!pip install wandb huggingface_hub optuna optuna-integration[wandb] -q

In [ ]:
# Clone repo
!rm -rf /content/PoT
!git clone https://github.com/Eran-BA/PoT.git /content/PoT
%cd /content/PoT
!git checkout feature/experiments

In [ ]:
# Check GPU
import torch
print(f"GPU: {torch.cuda.get_device_name()}")
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# W&B Login
import wandb
wandb.login()

In [ ]:
# Download Sudoku-Extreme dataset
import sys
sys.path.insert(0, '/content/PoT')
from src.data import download_sudoku_dataset

download_sudoku_dataset(
    output_dir='/content/PoT/data/sudoku-extreme-10k-aug-100',
    subsample_size=10000,
)
print("✓ Dataset ready!")

## Training

In [ ]:
# Train from scratch
!python scripts/train_sudoku_swin.py \
    --data-dir data/sudoku-extreme-10k-aug-100 \
    --epochs 1000 \
    --batch-size 768 \
    --lr 3.7e-4 \
    --h-cycles 2 \
    --l-cycles 6 \
    --halt-max-steps 5 \
    --max-depth 128 \
    --hrm-grad-style \
    --wandb \
    --project sudoku-swin \
    --device cuda

## Resume from W&B Checkpoint

# Resume training from W&B checkpoint
!python scripts/train_sudoku_swin.py \
    --data-dir data/sudoku-extreme-10k-aug-100 \
    --epochs 2000 \
    --batch-size 768 \
    --lr 1e-4 \
    --h-cycles 2 \
    --l-cycles 6 \
    --halt-max-steps 5 \
    --max-depth 128 \
    --hrm-grad-style \
    --wandb \
    --project sudoku-swin \
    --device cuda \
    --resume "wandb:YOUR_ENTITY/sudoku-swin/sudoku-swin-best:latest"

In [ ]:
## Optuna Depth Search

Find optimal depth configuration (h_cycles, l_cycles, halt_max_steps).

In [ ]:
# Run Optuna depth search
!python scripts/optuna_depth_search.py \
    --checkpoint "wandb:YOUR_ENTITY/sudoku-swin/sudoku-swin-best:latest" \
    --data-dir data/sudoku-extreme-10k-aug-100 \
    --n-trials 50 \
    --max-samples 100 \
    --max-depth 3000 \
    --wandb \
    --project sudoku-depth-search \
    --device cuda

In [ ]:
## Download Best Model

# Download to computer
from google.colab import files
files.download('checkpoints/swin/best_model.pt')

In [ ]:
# Or download from W&B Artifacts
import wandb
run = wandb.init(project="sudoku-swin")
artifact = run.use_artifact("sudoku-swin-best:best")
artifact_dir = artifact.download()
print(f"Downloaded to: {artifact_dir}")

In [ ]:
---
**Done!** Check your W&B dashboard for training curves and Optuna results.